# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
# module = os.path.abspath('/home/learner/DLA_project/src/main')
module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [3]:

manager.load_all()


Loading data from S2
	Path=C:\WESAD\S2\S2.pkl
Loading data from S3
	Path=C:\WESAD\S3\S3.pkl
Loading data from S4
	Path=C:\WESAD\S4\S4.pkl
Loading data from S5
	Path=C:\WESAD\S5\S5.pkl
Loading data from S6
	Path=C:\WESAD\S6\S6.pkl
Loading data from S7
	Path=C:\WESAD\S7\S7.pkl
Loading data from S8
	Path=C:\WESAD\S8\S8.pkl
Loading data from S9
	Path=C:\WESAD\S9\S9.pkl
Loading data from S10
	Path=C:\WESAD\S10\S10.pkl
Loading data from S11
	Path=C:\WESAD\S11\S11.pkl


In [ ]:
manager.compute_features();
manager.compute_features_stress();

conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6


In [ ]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)

In [ ]:
# Lets go ahead and reshape this data 
X1 = (   manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std'] )
X1 = np.reshape(X1, (len(manager.FEATURES['a_mean'])  ,15))

X2 = (   manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std'] )
X2 = np.reshape(X2, (len(manager.STRESS_FEATURES['a_mean'])  ,15))


In [ ]:
# import pip

# try:
#     import keras
# except ImportError:
#     ! sudo pip3 install keras
# try:
#     import h5py
# except ImportError:
#     ! sudo pip3 install h5py
# try:
#     import ibmiotf
# except ImportError:
#     ! sudo pip3 install ibmiotf
# try:
#     import tensorflow
# except ImportError:
#     ! sudo pip3 install tensorflow

In [ ]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, )

tf.set_random_seed(1)

session = tf.Session(graph=tf.get_default_graph(), config=session_conf)
keras.backend.set_session(session)

In [ ]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [ ]:
print(len(y1))
print(len(y2))

Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [ ]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

In [ ]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

Need to scale the data and hope we get better results.
1. fit the scaler to the training data (fit_transform)
2. transform training data with the scaler
3. fit the model with transformed data
4. transform test data with the scaler
5. predict using model and output of (4)

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [ ]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)
X_train_scaled = normalize(X_train)
X_test_scaled = normalize(X_test)

In [ ]:
# Should be

# input_shape=(number of sequences=?, time_steps=None, features=15)

# target=(number of sequences, time_steps, targets)

In [ ]:
X_train = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

In [ ]:
num_neurons = 16 # one more than the number of features in our dataset

In [ ]:
print('Building the LSTM network...')


model = Sequential()

model.add(LSTM(16, input_shape=(1, 15), return_sequences=True))
# model.add(LSTM(16, input_shape=(1, 15), dropout=0.35, recurrent_dropout=0.35, return_sequences=True))

# Note:
# Need to do return_sequences=False for the layer *before* dense
# https://stackoverflow.com/questions/51763983/error-when-checking-target-expected-dense-1-to-have-3-dimensions-but-got-array
model.add(LSTM(16, input_shape=(1, 15), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

print("inputs: " , model.input_shape)

print("outputs: ", model.output_shape)

print("actual inputs: ", np.shape(X_train))

print("actual outputs: ", np.shape(y_train))

In [ ]:
model.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


In [ ]:
print('Training LSTM...')

batch_size = 1 # I think I want to use batch_size = 1


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

In [ ]:
#manager.train_model()

# manager.test_model()
